In [1]:
PROJECT_PATH = "C:/Users/Thierry/Desktop/ProjetInterPromo/ProjectInterPromo/"
"""
Created on Mon Jan 10 11:02:55 2022
Group 2
@authors: L-A C
"""

'\nCreated on Mon Jan 10 11:02:55 2022\nGroup 2\n@authors: L-A C\n'

In [4]:
pip install pyod

Note: you may need to restart the kernel to use updated packages.
  Created wheel for pyod: filename=pyod-0.9.7-py3-none-any.whl size=136272 sha256=f719876e4940bcc3c9ec3a39e8c82046ed5b5c00931a1dda8c9ccb4a56a877e2
  Stored in directory: c:\users\thierry\appdata\local\pip\cache\wheels\7c\a1\43\7e69ab42be37151d91eee7daee202561a2fe01e2f48388b153
Successfully built pyod


# Importation

In [5]:
import pandas as pd
import plotly.express as px
import sweetviz as sv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pyod.models.knn import KNN
pd.set_option('max_columns', None)
%matplotlib inline

# Normalisation de toutes les données by Théo

In [21]:
df = pd.read_csv(PROJECT_PATH + 'data/CLEAN/donnee_v2_ech_train_transfo.csv').drop(columns="Unnamed: 0")

def normalize(df: pd.DataFrame):
    autre, bool_col = bool_other_col(df)

    y_col = ["TOP_FRAUDE", "TOP_FRAUDE_CARTE", "TOP_FRAUDE_VIREMENT"]

    autre_col = list(set(df.columns) - set(bool_col + y_col + ["ID"]))

    sub_df = df[autre_col]

    df[autre_col] = ((sub_df - sub_df.mean()) / sub_df.std())
    
    return df

# Variable à analyser

In [29]:
EQUIP = ['top_affecte', 'note_Funivers', 'note_Fassu', 'note_Fcredit', 'note_Fepargne', 'note_FService', 'top_produit_collecte', 'top_produit_monetaire_dispo', 'top_produit_monetaire_bloque', 'top_produit_emprunts', 'top_produit_epargne_logement', 'nb_DAV', 'nb_produit_Assurance', 'nb_produit_Cartes', 'nb_produit_Credit', 'nb_produit_Epargne', 'nb_produit_Services', 'top_DAV', 'top_produit_Assurance', 'top_produit_Cartes', 'top_produit_Credit', 'top_produit_Epargne', 'top_produit_Services']
DIG = ['INDICMAIL', 'INDICTEL', 'INDICPOR', 'INDICSMS', 'INDICMAILING', 'TOP_E_RELEVE', 'MESSAGERIE_WEB_ACTIVE', 'FIAB_TEL_DOMICILE', 'FIAB_TEL_TRAVAIL', 'FIAB_TEL_PORTABLE', 'FIAB_EMAIL', 'REFUS_SEA', 'TOT_CNX_WEB_WEEK_6M', 'TOT_CNX_WEB_END_6M', 'TOT_CCOMPTE_WEB_6M', 'TOT_IMPRESSIONRIB_WEB_6M', 'TOT_TLCHGMTEDOC_WEB_6M', 'TOT_CCOMPTE_MBANQUE_6M', 'TOT_CMS_WEB_6M', 'TOT_CNX_INVSTORE_WEB_6M', 'NB_JR_WEB_6M', 'NB_CNXS_WEB_6M', 'DUREE_CNX_WEB_TOT_6M', 'DUREE_CNX_MOY_WEB_6M', 'NB_JR_MBANQUE_6M', 'NB_CNXS_MBANQUE_6M', 'NB_JR_CABOURSE_6M', 'NB_CNXS_CABOURSE_6M', 'NB_JR_MACARTE_6M', 'NB_CNXS_MACARTE_6M', 'ALERTE_SMS_6M', 'NB_SAISI_IBAN_6M', 'NB_RDV_MBANQUE_6M', 'NB_RDV_WEB_6M', 'NB_PAP_RECU_6M', 'NB_PAP_CLIQUEE_6M', 'TAUX_LECTURE_PAP_6M', 'TOP_SECURIPASS' ]
HB = ['NBJ_CNT_AGENCE_DEPOT_6M', 'NBJ_CNT_AGENCE_VENTE_6M', 'NBJ_CNT_RDV_AGENCE_6M', 'NBJ_CNT_AUTOMATE_RETRAIT_6M', 'NBJ_CNT_AUTOMATES_DEPOT_6M', 'NBJ_CNT_AGENCE_RETRAIT_6M', 'NBJ_CNT_AGENCE_6M', 'NBJ_CNT_SMS_6M', 'NBJ_CNT_EMAIL_6M', 'NBJ_CNT_APPELENTRANTS_6M', 'NBJ_CNT_AUTOMATE_6M', 'NBJ_CNT_LETTRES_6M', 'TOT_CVAD_WEB_6M', 'TOT_VEL_WEB_6M', 'TOT_ATT_SIGN_VAD_WEB_6M', 'TOT_CHABITAT_WEB_6M', 'TOT_MODIF_PARAM_CARTE_WEB_6M', 'TOT_VIR_WEB_6M', 'TOT_VIR_M_6M', 'NB_VIR_AGENCE_6M', 'NB_RETRAIT_MN_6M', 'NBPAIEMENTS_6M', 'MNTPAIEMENTS_6M', 'NBPAIEMENTS_WEB_6M', 'MNTPAIEMENTS_WEB_6M', 'NB_PAIEMENT_SS_CONTACT_6M', 'MNT_PAIEMENT_SS_CONTACT_6M', 'TX_PAIEMTNB_WEB_TOT_6M', 'TX_PAIEMTMTT_WEB_TOT_6M', 'NB_VEL_EPARGNE_6M', 'NB_VEL_PACIFICA_6M', 'NB_MAIL_RECU_6M', 'NB_MAIL_LU_6M', 'TAUX_LECTURE_EMAIL_6M', 'NB_MS_RECU_6M', 'NB_MS_LU_6M', 'TAUX_LECTURE_MS_6M', 'NB_DEPOSE_PREDICA_WEB_6M', 'NB_DEPOSE_PREDICA_AG_6M', 'NB_DEPOSE_PACIFICA_AG_6M', 'NB_DEPOSE_EPARGNE_WEB_6M', 'NB_DEPOSE_EPARGNE_MBQE_6M', 'NB_DEPOSE_EPARGNE_AG_6M', 'NB_DEPOSE_PACIFICA_WEB_6M', 'NB_VENTE_DEVIS_PACIFICA_WEB_6M', 'NB_VENTE_DEVIS_PACIFICA_AG_6M', 'NB_DEVIS_PACIFICA_WEB_6M', 'NB_DEVIS_PACIFICA_AG_6M', 'NB_VENTE_DEVIS_PACIFICA_6M', 'NB_PAIMT_PAYLIB_6M', 'MTT_PAIMT_PAYLIB_6M', 'top_entretien_3M', 'top_entretien_6M', 'top_entretien_12M', 'nb_mois_dernier_entretien', 'note_mire', 'note_FOpDebitrice', 'NB_MOY_RETRAIT_M_CARTE_P', 'SGMT_FIDELITE', 'nb_pret_realise', 'mt_pret_realise', 'nb_mois_dern_realisation_pret', 'top_real_pret_3M', 'top_real_pret_6M', 'mt_op_cred_ep_a_terme_M', 'mt_op_cred_ep_a_terme_M6', 'mt_op_debit_ep_a_terme_M', 'mt_op_debit_ep_a_terme_M6', 'mt_operation_credit_depot_M', 'mt_operation_credit_depot_M6', 'mt_operation_credit_ep_a_vue_M', 'mt_operation_credit_ep_a_vue_M6', 'mt_operation_debit_depot_M', 'mt_operation_debit_depot_M6', 'mt_operation_debit_ep_a_vue_M', 'mt_operation_debit_ep_a_vue_M6', 'mt_paiement_carte_M', 'mt_paiement_carte_M6', 'mt_paiement_chq_M';, 'mt_paiement_chq_M6', 'mt_pmts_carte_etr_M', 'mt_pmts_carte_etr_M6', 'mt_prlv_impaye', 'mt_remise_chq_M', 'mt_remise_chq_M6', 'mt_ret_carte_Aut_bq_M', 'mt_ret_carte_Aut_bq_M6', 'mt_ret_carte_CRCA_M', 'mt_ret_carte_CRCA_M6', 'mt_ret_carte_etr_M', 'mt_ret_carte_etr_M6', 'mt_retrait_carte_M', 'mt_retrait_carte_M6', 'nb_cheque_impaye', 'nb_commission_intervention', 'nb_ferm_prod_6M', 'nb_flux_cred_normeB2_M', 'nb_flux_cred_normeB2_M6', 'nb_irregularite_particuliere', 'nb_op_cred_ep_a_terme_M', 'nb_op_cred_ep_a_terme_M6', 'nb_op_debit_ep_a_terme_M', 'nb_op_debit_ep_a_terme_M6', 'nb_operation_credit_depot_M', 'nb_operation_credit_depot_M6', 'nb_operation_credit_ep_a_vue_M', 'nb_operation_credit_ep_a_vue_M6', 'nb_operation_debit_depot_M', 'nb_operation_debit_depot_M6', 'nb_operation_debit_ep_a_vue_M', 'nb_operation_debit_ep_a_vue_M6', 'nb_ouv_prod_6M', 'nb_paiement_carte_M', 'nb_paiement_carte_M6', 'nb_paiement_chq_M', 'nb_paiement_chq_M6', 'nb_pmts_carte_etr_M', 'nb_pmts_carte_etr_M6', 'nb_prlv_impaye', 'nb_remise_chq_M', 'nb_remise_chq_M6', 'nb_ret_carte_Aut_bq_M', 'nb_ret_carte_Aut_bq_M6', 'nb_ret_carte_CRCA_M', 'nb_ret_carte_CRCA_M6', 'nb_ret_carte_etr_M', 'nb_ret_carte_etr_M6', 'nb_retrait_carte_M', 'nb_retrait_carte_M6', 'top_debiteur_M', 'top_ferm_prod_6M', 'top_ouv_prod_6M', 'mt_factu_ttc_M', 'mt_factu_ttc_M6', 'mt_reduc_ttc_M', 'mt_reduc_ttc_M6', 'mt_rist_ttc_M', 'mt_rist_ttc_M6', 'Fidelite_calcule', 'Flag_bq_principale']

SyntaxError: invalid syntax (<ipython-input-29-02e20c245d14>, line 3)

# Echantillonage

In [14]:
# Lecture du fichier de donnée
data_v2 = df

print("Nombre de fraude :", sum(data_v2["TOP_FRAUDE"]))
print("Nombre de données : ", len(data_v2))
print("Ratio : ", sum(data_v2["TOP_FRAUDE"]) / len(data_v2) * 100)

# Suppression des variables qualitative
quali_col = []
count = 0
for i, name in zip(data_v2.dtypes, data_v2.columns):
    if i == "object":
        quali_col.append(name)

data_quanti = data_v2.drop(columns=quali_col)

# 
data_quanti = data_quanti.sample(100000)

print("------------ Sur un echantillion ------------")
print("Nombre de fraude :", sum(data_quanti["TOP_FRAUDE"]))
print("Nombre de données : ", len(data_quanti))
print("Ratio : ", sum(data_quanti["TOP_FRAUDE"]) / len(data_quanti) * 100)

Nombre de fraude : 1461
Nombre de données :  764471
Ratio :  0.19111254710773856
------------ Sur un echantillion ------------
Nombre de fraude : 202
Nombre de données :  100000
Ratio :  0.202


# Modele KNN

In [26]:
# Modele de prediction qui retourne un score
data_EQUIP = df[EQUIP]
data_DIG = df[DIG]
data_HB = df[HB]
outliers_fraction = 0.01
clf = KNN(contamination = outliers_fraction)
xx,yy = np.meshgrid(np.linspace(0,1,100),np.linspace(0,1,100))
plt.figure(figsize=(20,15))
plt.show()

<Figure size 2000x1500 with 0 Axes>

### EQUIP

In [27]:
clf.fit(data_EQUIP)

KNN(algorithm='auto', contamination=0.01, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [28]:
score = clf.decision_function(data_EQUIP)
score
set(score)

{0.0,
 1.0,
 0.5066498560432251,
 0.31796844232223975,
 1.2594334746190725,
 2.20240570740134,
 0.8221235358324748,
 0.748473901797297,
 0.6163067170593308,
 0.8395624064281151,
 1.043008707156389,
 2.183148452737953,
 0.5831814597869248,
 0.633434298127309,
 1.100524040257369,
 0.7523023990108515,
 0.6037239426152257,
 0.569923816176207,
 0.7688221413938765,
 1.0262000121440098,
 0.6474243196128707,
 0.749192567280645,
 0.7028416476336337,
 0.803190492299493,
 0.8137107481529005,
 1.6243766037772933,
 0.6626632454258724,
 0.65682867401986,
 0.8144543208777831,
 0.9808503629834682,
 0.7145852827985741,
 0.9530273382685059,
 1.156462344898167,
 1.2329344701671594,
 1.4257066804208385,
 0.7547252040866121,
 0.9882871161724731,
 1.1179804465219203,
 0.9095141726340094,
 0.6993144736863192,
 0.7822646869401679,
 0.8541988170229771,
 1.1561933505193038,
 0.9408361839763281,
 0.7932951906772111,
 0.7381943186460178,
 0.6223495435947173,
 0.9326128212446549,
 1.1813229029244015,
 0.5750684597

In [30]:
value, f1_score, precision, recall = evaluation(score, data_quanti["TOP_FRAUDE"], display=True)

# L'air sous la courbe
print("Air sous la courbe : ", value)

NameError: name 'evaluation' is not defined

### DIG

In [ ]:
clf.fit(data_DIG)

In [7]:
score = clf.decision_function(data_DIG)
score
set(score)

In [ ]:
value, f1_score, precision, recall = evaluation(score, data_quanti["TOP_FRAUDE"], display=True)

# L'air sous la courbe
print("Air sous la courbe : ", value)

In [ ]:
clf.fit(data_HB)

In [ ]:
score = clf.decision_function(data_HB)
score
set(score)

In [ ]:
pyod.models.abod.ABOD

In [ ]:
pyod.models.cblof.CBLOF

In [ ]:
pyod.models.cof.COF

In [ ]:
pyod.models.iforest.IFOREST

In [ ]:
pyod.models.hbos.HBOS

In [ ]:
pyod.models.knn.KNN

In [ ]:
pyod.models.lof.LOF

In [ ]:
pyod.models.ocsvm.OCSVM

In [ ]:
pyod.models.pcs.PCA

In [ ]:
pyod.models.mcd.MCD

In [ ]:
pyod.models.sod.SOD

In [ ]:
pyod.models.sos.SOS

In [ ]:
pyod.models.xgbod.XGBOD

In [ ]:
pyod.models.rod.ROD

In [ ]:
pyod.models.loci.LOCI

In [ ]:
pyod.models.lmdd.LMDD

In [ ]:
pyod.models.fastABOD.FASTABOD